# IBM Applied Data Science Capstone - Report

This report outlines the works completed and key findings arising from the capstone project for the Data Science Professional Certificate from IBM on Coursera.

## Table of contents

1. [Introduction](#introduction)

2. [Data](#data)

3. [Methodology](#methodology)

5. [Results](#results)

6. [Discussion](#discussion)

7. [Conclusion](#conclusion)

8. [References](#references)

## 1. Introduction<a name='introduction'></a>

Toronto's vegetarian and vegan scene has come a long way in the past few years and continues to grow.

With the consumption of healthy and ethical food being on the rise, it’s a great time to be opening a vegetarian or vegan restaurant. 
The objective of this project is to identify suitable neighborhoods for opening a **vegetarian/vegan restaurant** in **Toronto, Canada**. 

We would want to find neighborhoods with the following characteristics:
1. Low direct competition and that are not already crowded with vegan/vegetarian restaurants.
2. Moderate to high population density of young adult population. According to a 2018 Dalhousie University study, 2 out of 3 vegans in Canada are either from generation Z or millennials, so this population group will be the main target market.[1]
3. Moderate expandable income.
4. Located in the 'Old Toronto' district, so it could also benefit from tourist traffic. 

Specifically, this report can be used by any stakeholder that is interested in gaining analytics insights for choosing a suitable location to open a vegetarian or vegan restaurant in the city of Toronto.


## 2. Data<a name='data'></a>

Data were collected from multiple sources. Description of each data and their respective data sources are discussed below. 

* **City of Toronto Neighbourhood Profiles(CSV):** The 2016 Neighbourhood Profiles are based on data collected by Statistics Canada in its 2016 Census of Population and are accesible in the City of Toronto Open Data Portal. This data source was used to extract data on total population density aged 15-54 per square meter, and the average after-tax income of each neighborhood.[2]

* **Venues in Toronto neighborhoods:** Data on vegan/vegetarian/veg-option restaurants in every neighborhood, and their locations was obtained from the Foursquare site by using the Foursquare API.[3]

* **Boundaries of City of Toronto Neighbourhoods(GeoJSON):** The files contains standard geospatial data and geographic features for the 140 neighborhoods in Toronto. This was crucial for map visualization.[4]


## 3. Methodology <a name='methodology'></a>

In this project, our efforts were directed on detecting neighborhoods of Toronto that could be suitable for opening a vegetarian/vegan restaurant. The factors that influenced the selection of suitable neighborhoods were the following:

* the number of vegetarian/vegan/veg-option restaurants in the neighborhood
* the total population density per square meter aged 15-54 in the neighborhood
* the average after-tax income of the neighborhood

Based on these feautures, cluster analysis was employed to look for groups of neighborhoods that present similar factors; specifically, we employed the k-means clustering, which is a partitioning method. 

### 3.1 Data Collection and Preprocessing

The first phase of our analysis consisted in collecting the data required, which came from multiple sources, and in preprocessing the data so it could be suitable for cluster analysis. The three main Python libraries that were used for data preprocessing were pandas, numpy and geopandas. 

Next, we will discuss how the three main datasets that were used on this project were pre-processed.

#### 3.1.1  Socioeconomic data for each neighborhood in the city of Toronto

The socio-economic data of interest was the total population density aged 15-54 and the average after-tax income of each neighborhood. These data was part of a larger dataset in CSV format, so many methods in the Pandas library such as slicing, renaming and NumPy methods such as replacing, rounding were used.

Specifically, in order to get the total population density per square meter aged 15 to 54, the total population aged 15-24 and the total population aged 25-54 were added, and divided it by the land area of each neighborhodd in square meters. 

#### 3.1.2. Boundaries and coordinates of the city of Toronto neighborhoods

This geoJSON file was converted to a geopandas dataframe and was used for all map visualizations. We also extracted the longitude and latitude values of the center's of each neighborhood as well as the neighbourhood boundaries that were represented as polygon objects.

The city of Toronto has a total of 140 neighborhoods, however for the porpuse of this analysis, we were only interested in exploring the 44 neighborhoods that are part of the "Old Toronto" ditrict, wich are the neighborhoods closer to the city center.
    
The Folium library was used to create an interactive map showing the boundaries and the center's of the 44 neighborhoods in the 'Old Toronto' district as shown in Fig. 1. 

<img src="Screenshots/Map of Toronto and its 44 NBHD.PNG">

#### 3.1.3. Data on vegan/vegetarian/veg-options restaurants in each neighborhood

Having the coordinates of our location candidates, resquests were sent to the explore endpoint of the Foursquare API to get data on the vegan/vegetarian restaurants that are within a radius of 2.5 km of the center of each neighborhood. The Category ID corresponding to "Vegetarian/Vegan restaurant" was taken from Foursquare web site.[3]

Since the neighborhoods vary in size of land areas, the radius was set big enough to make sure that all existing "vegan/vegetarian" restaurants in each neighborhood were returned, even for the biggest neighborhoods. However, this caused that requests associated to small neighborhoods returned restaurants that were located in adjacents neighborhoods. Out of 732 returned venues, only 110 venues had an unique ID.

In order to remove all duplicates, it was needed to identify in which neighborhood each venue really belonged to. This was achieved by using the shapely package to test if the geographical coordinates of each venue, represented as a Point object, was inside (True) or outside (False) the boundaries of each neighborhood, represented as Polygon objects. 

Also, the Foursquare API returned 12 venues which primary categories were not "Vegetarian/Vegan" restaurants, which were further explore using the venue endpoint of the Foursquare API. These venues were vegan-friendly but not fully vegetarian/vegan venues. From those venues, proper restaurants were kept in the dataset, while those that are not proper restaurants such as bars, pubs, and coffe shops were removed since they are not considered direct competitors.

The Folium library was used to create an interactive map showing the location of all final venues as shown in Fig. 2. 

<img src="Screenshots/Map of Venues.PNG">

Next, the number of restaurants were counted by neighborhoods and added to the final dataframe. NaN values associated to those neighborhoods with no restaurants were replaced by zero value. Note that all datasets were merged using the 'Neighborhood ID' as the primary key. 

#### 3.1.4. Final Pre-processing

Prior to performing the cluster analysis, we ensured that the data was suitable for cluster analysis. The folowing steps were performed:

1. Checked that the size of the sample (n) was succiently large to ensure signicant partitions. A minimum sample size of 2^p , where p represents the clustering variables, is recommended as a rough guideline. Our sample size of 44 > 2^3 = 8 was appropriate to perform the
cluster analysis.
2. Checked collinearity among the clustering variables, since a high degree of collinearity among the clustering variables could be problematic since the variables are not sufficiently distinctive to classify different groups. No two clustering variables present high correlation ( |r| > 90) among themselves.
3. Normalized clustering variables since data set contains numerical features that have been measured in different units.


### 3.2 Model Fitting: k-Means Clustering

The implementation of the k-means clustering in Python from the package scikit-learn was used to group the neighborhoods into *k* clusters suh that all neighborhoods with similar socioeconomic characteristics and number of vegetarian/vegan restaurants are in the same cluster. 

Since the number of clusters present in this dataset (k) was unknown, the Elbow method was used to identify the optimal value of k. ThePython package, kneed, was also used to identify the elbow point programmatically. As shown in Figure 3, the optimum k selected by the Elbow method was 4.

<img src="Screenshots/Elbow Method.PNG">

## 4. Results <a name='results'></a>

Based on the outcome of k-means clustering, the 44 neighborhoods in Toronto were grouped into four distinctive clusters. Fig 4 provides a final representation of the final cluster membership. 

<img src="Screenshots/Map of Cluster Membership.PNG">

70% (31/44) of the neighborhoods were grouped into one cluster, Cluster 1. There are 4 neighborhoods in Clusters 0 and Clusters 2, and 5 in Cluster 3.

In order to create a profile for each group, the centroids of the clusters were examined using a Parallel Coordinates Plot, as shown in Fig. 5. 

<img src="Screenshots/Centroids Parallel Coordinates Plot.PNG">

A profile was created for each group, considering the common characteristics of each cluster. The 4 clusters can be described as follows, ordered from most suitable to least:

* Cluster 0: Most Suitable neighborhoods: It's made up of 4 neighborhoods. High target population density, moderate income level, low competition.
* Cluster 1: Suitable neighborhoods: It's made up of 31 neighborhoods. Moderate target population density, moderate despensable income, low competition.
* Cluster 2: Semi- suitable neighborhoods: It's made up of 4 neighborhoods. Low target population, high despensable income, low competition.
* Cluster 3: Least suitable neighborhoods: It's made up of 5 neighborhoods. High target population, moderate despensable income, high competition.

## 5. Discussion <a name='discussion'></a>

Based on the analysis above, neighborhoods in Cluster 0: Church-Yonge Corridor, Mount Pleasant West, North St. James Town, and Regent Park are recommended for opening a vegetarian/vegan restaurant in Toronto. These neighborhoods have low number of Vegetarian restaurants making competition easier than in other neighborhoods. They also count with high target population size. This is important because vegetarian/vegan restaurants often need every customer they can get to stay afloat. Specifically, Church-Yonge Corridor is closed to the city center and could attract tourist looking for vegetarian/vegan options.

It's important to note that recommended neighborhoods should be considered only as a starting point for more detailed analysis taking into account other factors. Also, all of the above analysis is depended on the adequacy and accuracy of Four Square data. A more comprehensive analysis and future work would need to incorporate data from other external databases.


## 6. Conclusion <a name='conclusion'></a>

The objective of this project was to identify Toronto neighborhoods with low number of vegetarian/vegan restaurants, with moderate to high population density of young adult population and moderate dispensable income, in order to aid stakeholders in narrowing down the search for optimal locations for a new vegetarian/vegan restaurant. 

Clustering of neighborhoods was performed in order to create groups of neighborhoods with similar socioeconomic characteristics and number of vegetarian/vegan restaurants. The final recommended neighborhoods can be used as starting points for final exploration by stakeholders.

## 7. References <a name='references'></a>

[1] CTV News Article: More than 3 million Canadians vegetarian or vegan: study

[2] City of Toronto’s Open Data Portal 

[3] Foursquare. Places API

[4] City of Toronto’s Open Data Portal
